In [1]:
!wget https://s3.amazonaws.com/assets.datacamp.com/blog_assets/fake_or_real_news.csv

--2019-11-05 07:24:21--  https://s3.amazonaws.com/assets.datacamp.com/blog_assets/fake_or_real_news.csv
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.184.253
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.184.253|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 30696129 (29M) [text/csv]
Saving to: ‘fake_or_real_news.csv’

fake_or_real_news.c 100%[===================>]  29.27M  53.4MB/s    in 0.5s    

2019-11-05 07:24:22 (53.4 MB/s) - ‘fake_or_real_news.csv’ saved [30696129/30696129]



In [2]:
!pip install ktrain

     |████████████████████████████████| 174kB 3.4MB/s 
     |████████████████████████████████| 317kB 46.5MB/s 
     |████████████████████████████████| 112kB 48.4MB/s 
     |████████████████████████████████| 1.0MB 45.3MB/s 
     |████████████████████████████████| 204kB 48.5MB/s 
     |████████████████████████████████| 1.8MB 29.9MB/s 
  Created wheel for ktrain: filename=ktrain-0.5.2-cp36-none-any.whl size=213552 sha256=c7db20b6a7e66933693364bbbb1fb362c4c7a54ca3e0c415a9ae03d9a11f047e
  Stored in directory: /root/.cache/pip/wheels/72/71/d8/54ea825a57a816701162b9b0696765593182ce37d9f30d7819
  Created wheel for keras-bert: filename=keras_bert-0.80.0-cp36-none-any.whl size=37923 sha256=0277860273bffa1a1157c57c024c9e3a5b296ed02cf0813bc00205ad43f0220f
  Stored in directory: /root/.cache/pip/wheels/63/dc/87/3260cb91f3aa32c0f85c5375429a30c8fd988bbb48f5ee21b0
  Created wheel for seqeval: filename=seqeval-0.0.12-cp36-none-any.whl size=7424 sha256=cf9075261759c83ba52cc45654051ce30118be3354584cacf7c

In [0]:
import pandas as pd

In [0]:
data=pd.read_csv('/content/fake_or_real_news.csv')

In [5]:
data.head()

,Unnamed: 0,title,text,label
0,8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE
1,10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE
2,3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL
3,10142,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE
4,875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL


In [0]:
dummy=pd.get_dummies(data['label'])

In [0]:
data=pd.concat([data,dummy],axis=1)

In [9]:
data.head()

,Unnamed: 0,title,text,label,FAKE,REAL
0,8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE,1,0
1,10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE,1,0
2,3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL,0,1
3,10142,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE,1,0
4,875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL,0,1


In [0]:
data=data.drop(['label','title','Unnamed: 0'], axis=1)

In [11]:
data.head()

,text,FAKE,REAL
0,"Daniel Greenfield, a Shillman Journalism Fello...",1,0
1,Google Pinterest Digg Linkedin Reddit Stumbleu...,1,0
2,U.S. Secretary of State John F. Kerry said Mon...,0,1
3,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",1,0
4,It's primary day in New York and front-runners...,0,1


In [0]:
df=data

In [12]:
import ktrain
from ktrain import text


Using TensorFlow backend.


using Keras version: 2.2.4


In [16]:
(x_train,y_train),(x_test,y_test),preproc=text.texts_from_df(df,'text',
                                                                 label_columns=['FAKE', 'REAL'],
                                                                   maxlen=75, 
                                                                   max_features=100000,
                                                                   preprocess_mode='bert',
                                                                   val_pct=0.1)

downloading pretrained BERT model (uncased_L-12_H-768_A-12.zip)...
[██████████████████████████████████████████████████]
extracting pretrained BERT model...
done.

cleanup downloaded zip...
done.

preprocessing train...
language: en


preprocessing test...
language: en


In [17]:
model=text.text_classifier('bert',(x_train,y_train),preproc=preproc)

Is Multi-Label? False
maxlen is 75
done.


In [0]:
learner=ktrain.get_learner(model,train_data=(x_train,y_train),val_data=(x_test,y_test),batch_size=32)

In [19]:
learner.fit_onecycle(2e-5, 1)



begin training using onecycle policy with max lr of 2e-05...
Train on 5701 samples, validate on 634 samples
Epoch 1/1
5701/5701 [==============================] - 5123s 899ms/step - loss: 0.2883 - acc: 0.8683 - val_loss: 0.1698 - val_acc: 0.9369


In [0]:
predictor = ktrain.get_predictor(learner.model, preproc)


In [24]:
predictor.predict(data['text'][1])


'FAKE'